# 4. Comprehensive EDF Signal Inspection (v3)

**Objective:** To definitively identify the signals and events contained within each type of `.edf` file. This version uses file-specific loading logic to correctly handle both signal and event files and generates a clean, informative plot.

## 4.1 Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import mne
from IPython.display import display, Markdown

# Add src directory to Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data_loader import load_edf_data

# Configure plotting
plt.rcParams['figure.figsize'] = (20, 14)
pd.set_option('display.max_rows', 100)

## 4.2 Intelligent Data Loading

This cell inspects each file. It loads signal files, cleans them by dropping checksum columns, and specifically loads the `EVE` file as annotations.

In [ ]:
# --- Configuration ---
DATA_DIR = '../data/2025/'
FILE_PATHS = {
    'BRP': os.path.join(DATA_DIR, '20250617_023551_BRP.edf'),
    'EVE': os.path.join(DATA_DIR, '20250617_023547_EVE.edf'),
    'PLD': os.path.join(DATA_DIR, '20250617_023551_PLD.edf'),
    'CSL': os.path.join(DATA_DIR, '20250617_023547_CSL.edf')
}
PLOT_DURATION_S = 30

# --- Data Loading ---
loaded_signals = {}
annotations = None

for key, path in FILE_PATHS.items():
    display(Markdown(f"### Processing: `{key}` file"))
    if not os.path.exists(path):
        display(Markdown(f"**Status:** File not found."))
        print('-'*60)
        continue
        
    if key == 'EVE':
        try:
            annotations = mne.read_annotations(path)
            # Get the start time from the file name to offset annotations
            fname = os.path.basename(path)
            start_time_str = fname.split('_')[0] + fname.split('_')[1]
            session_start_time = pd.to_datetime(start_time_str, format='%Y%m%d%H%M%S')
            annotations.onset += session_start_time.timestamp()
            display(Markdown(f"**Success:** Loaded {len(annotations)} events/annotations."))
            display(annotations[:5])
        except Exception as e:
            display(Markdown(f"**Failed:** Could not read annotations: {e}"))
    else:
        try:
            df = load_edf_data(path)
            if df is not None and not df.empty:
                # Clean the data: drop checksum columns
                cols_to_drop = [c for c in df.columns if 'Crc' in c]
                df.drop(columns=cols_to_drop, inplace=True)
                
                if not df.empty:
                    loaded_signals[key] = df
                    display(Markdown(f"**Success:** Loaded as Signal data."))
                    display(Markdown(f"**Signals found:** {list(df.columns)}"))
                    display(df.head())
                else:
                    display(Markdown(f"**Info:** File contained only checksum signals."))
            else:
                display(Markdown(f"**Info:** No data loaded from file."))
        except Exception as e:
            display(Markdown(f"**Failed:** Could not load as signal data: {e}"))
            
    print('-'*60)


## 4.3 Plotting Signals and Events

This plot shows each loaded signal group in its own subplot for clarity. Events from the `EVE` file are overlaid on all plots.

In [ ]:
if loaded_signals:
    # Determine the common time range for plotting
    first_timestamp = min(df.index.min() for df in loaded_signals.values())
    plot_start_time = first_timestamp
    plot_end_time = plot_start_time + pd.Timedelta(seconds=PLOT_DURATION_S)
    
    num_axes = len(loaded_signals)
    fig, axes = plt.subplots(num_axes, 1, figsize=(20, 4 * num_axes), sharex=True)
    fig.suptitle(f'First {PLOT_DURATION_S} Seconds of All Signals with Events', fontsize=16)
    
    for i, (key, df) in enumerate(loaded_signals.items()):
        ax = axes[i] if num_axes > 1 else axes
        plot_data = df.loc[plot_start_time:plot_end_time]
        
        plot_data.plot(ax=ax, legend=True)
        ax.set_title(f'{key} Signals', fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.legend(loc='upper right')
        
        # Overlay events on this axis
        if annotations is not None:
            for ann in annotations:
                onset = pd.to_datetime(ann['onset'], unit='s')
                duration = pd.to_timedelta(ann['duration'], unit='s')
                if plot_start_time <= onset < plot_end_time:
                    ax.axvspan(onset, onset + duration, color='#d62728', alpha=0.2, zorder=0, label='_nolegend_')
                    ax.text(onset, ax.get_ylim()[1] * 0.95, ann['description'], rotation=75, verticalalignment='top', fontsize=10)
    
    plt.xlabel('Time', fontsize=12)
    plt.xlim(plot_start_time, plot_end_time)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()
else:
    print("No signal data was successfully loaded to plot.")